# Gender prediction

From [Kaggle](https://www.kaggle.com/code/shahraizanwar/age-gender-ethnicity-prediction)

In [ ]:
import numpy as np 
import pandas as pd
# import tensorflow as tf
# import tensorflow.keras.layers as L
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('../data/age_gender.csv')

data['pixels'] = data['pixels'].apply(lambda x: np.array(x.split(), dtype = 'float32'))
data['pixels'] = data['pixels'].apply(lambda x:x/255)

gender_dist = data['gender'].value_counts().rename(index={0:'Male', 1:'Female'})

data.head(5)

In [ ]:
print(f'Total rows: {len(data)}')
print(f'Total columns: {len(data.columns)}')

In [ ]:
X = np.array(data['pixels'].tolist())
X = X.reshape(len(data), 48, 48) # reshape each array of length 48x48 into matrix
y = data['gender'].tolist()

In [ ]:
# Plot some faces
X = X.reshape(X.shape[0], 48,48, 1)
plt.figure(figsize=(16,16))
for i in range(1500,1520):
    plt.subplot(5,5,(i%25)+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(data['pixels'].iloc[i].reshape(48,48))
    plt.xlabel(
        "Age:"+str(data['age'].iloc[i])+
        "  Ethnicity:"+str(data['ethnicity'].iloc[i])+
        "  Gender:"+ str(data['gender'].iloc[i])
    )
plt.show()

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision

In [ ]:
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
from math import floor

train_ratio = 0.7 # 70% train, 30% test
slice = int(floor(len(X) * train_ratio))

class DataSetLoader(Dataset):
    def __init__(self, x, y):       
        self.x = x
        self.y = y
        
        self.x_train = torch.tensor(self.x, dtype=torch.float32)
        self.y_train = torch.tensor(self.y)
    
    def __len__(self):
        return len(self.y_train)
    
    def __getitem__(self, idx):
        img = self.x_train[idx]
        img = img.unsqueeze(0)
        
        return img, self.y_train[idx]
    
trainset = DataSetLoader(X[:slice], y[:slice])
testset = DataSetLoader(X[slice:], y[slice:])

In [ ]:
batch_size = 64

trainloader = torch.utils.data.DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)

testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)

classes = ('male', 'female')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1)
        self.fc1 = nn.Linear(6400, 64)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(64, 2)
        
    def forward(self, x):
        # Convolutional layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
#         x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = x.reshape(x.shape[0], -1)
        
        # Fully connected layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x    

cf = Classifier().to(device=device)
cf

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cf.parameters(), lr=0.001, momentum=0.9)

In [ ]:
NUM_EPOCHS = 40
PLOT_LOSS = True
epochs = np.arange(1,NUM_EPOCHS+1)
losses = []

for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # model is in GPU, thus we send data also to GPU
        inputs, labels = inputs.to(device), labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = cf(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        loss_batch = running_loss / 2000
        if i % 200 == 199:    # print every 2000 mini-batches
            print(f'[Epoch: {epoch + 1}] Loss: {loss_batch}')
            running_loss = 0.0
    losses.append(loss_batch)

print('Finished Training')

In [ ]:
if PLOT_LOSS:
    plt.plot(epochs, losses)
    plt.show()

In [ ]:
# Save model
PATH = './models/gender_test.pth'
torch.save(cf.state_dict(), PATH)

In [ ]:
# Load model
PATH = './models/gender_test.pth'
cf = Classifier().to(device=device)
cf.load_state_dict(torch.load(PATH))

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

outputs = cf(images)

_, predicted = torch.max(outputs, 1)

# The outputs are energies for the 10 classes. The higher the energy for a class,
# the more the network thinks that the image is of the particular class.
# So, let’s get the index of the highest energy

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

In [ ]:
# Let us look at how the network performs on the whole dataset

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        # calculate outputs by running images through the network
        outputs = cf(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct // total} %')

In [ ]:
# what are the classes that performed well, and the classes that did not perform well?

# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        
        outputs = cf(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')